Chapter 2.  Proof Basics
================

In **Prove-It**, proving an **expression** to be a true statement under some set of assumptions creates a **judgment** object and a **proof** object.  **Prove-It**, in its core, is agnostic to intrinsic truth-aptness of **expressions** except when they are proven and become "judgments".  The `proveit.logic` package defines the Boolean concept which may be used for making statements of the form $A \in \mathbb{B}$ (where $\mathbb{B}$ represents the Boolean set of True or False), but this is outside **Prove-It**'s core.  As far as the core is concerned, an **expression** is either proven or not. If an expression is not proven, it may or may not be truth-apt (*i.e.*, it may or may not be able to be shown to be true under some set of assumptions).

A **judgment** contains assumptions, as **expressions**, and the single **expression** that is known to be a true statement under these assumptions.  It also has an associated **proof**.  The associated **proof** may get updated over time with a stronger proof (fewer assumption requirements) or newer proof that is as strong and has fewer proof steps.  A **proof** may also be disabled (via its `disable` method) to force an alternate or avoid its use.  Conveniently, a **judgment** object accesses the methods of the underlying proven **expression** seamlessly.  Using Python tricks (specifically, overloading the `__getattr__` and `__dir__` methods), the **judgment** behaves in most ways as if it were the known true **expression** it is wrapping.  This is useful because **expression** objects typically have convenience methods that are specific to the kind of expression, but it is also nice to display the object as a **judgment** (discussed next).

A **judgment** with some set of assumptions ${\cal A}$ is displayed in the form ${\cal A} \vdash S$ (or simply $\vdash S$ if ${\cal A}$ is empty), where $S$ is the (conditionally) true statement.  This single turnstile $\vdash$ is **Prove-It**'s indication that the **expression** on the right is derivable from the assumptions on the left.

For example, let us produce a trivial proof of the form $X \vdash X$:

In [ ]:
from proveit._common_ import fx
%begin proof_basics
# fx is f(x) which is simply an Operation with a Variable operator and a Variable operand,
# but that is irrelevant.  It was chosen arbitrarily.
fx.expr_info()

In [ ]:
fx_truth = fx.prove(assumptions=[fx])

The `prove` method attempts to prove that the **expression** is a true statement under the given set of assumptions and returns the **judgment** object if it is successful.  Automation strategies may be employed in the process and will be discussed in the <a href="tutorial08_automation.ipynb">automation</a> chapter.  In this case, $f(x)$ is trivially true if we use $f(x)$ as an assumption.  Again, **Prove-It** is agnostic to whether or not $f(x)$ is truth-apt.  This simply says that if $f(x)$ is assumed to be a true statement, then $f(x)$ is a true statement.  If `prove` is not successful, it raises a `ProofFailure` exception.

In [ ]:
from proveit import ProofFailure
try:
    fx.prove()
except ProofFailure as e:
    print("EXPECTED ERROR: ", e)

To obtain the **expression** from the **judgment**, and be able to display it as the **expression** only, access the `expr` attribute:

In [ ]:
fx_truth.expr

Its assumptions are accessible from its `assumptions` attribute (as a tuple).

In [ ]:
fx_truth.assumptions

But note that we may use the `Judgment` object as if it were the `Expression` object for most practical purposes.  It aquires all of the `Expression` object's attributes, except for some that it has overridden (such as `__class__`, `latex`, and `string`).

In [ ]:
print(dir(fx_truth))

In [ ]:
print(dir(fx_truth.expr))

In [ ]:
fx_truth.operands

In [ ]:
print(fx_truth.__class__) # this is a giveaway that it is not an actual Expression

In [ ]:
print(fx_truth.expr.__class__) # here it is

### Proof objects

Call the `proof` method of the **Judgment** to access the associated **Proof** object that proves the judgment.  A **Proof** is a DAG (directed, acyclic graph) of derivation steps that connect the judgment to *axioms*, *theorems*, or *assumptions*.

In [ ]:
fx_truth.proof()

Also note that the turnstile ($\vdash$) is hyperlinked to a notebook page that, when executed, will display the proof.  For more complicated proofs, you can also click on the turnstile of one of the proof steps to see the sub-proof for that step which can be a convenient way to explore a proof and dig into just the details of interest.

The **Judgment** that is proven by a **Proof** object is accessible via the `proven_truth` attribute.

In [ ]:
fx_truth.proof().proven_truth

The required sub-proofs are accessible via `required_proofs`.  In this example, this is an empty list.  $\{f(x)\} \vdash f(x)$ has no requirements.

In [ ]:
fx_truth.proof().required_proofs # no requirements for {f(x)} |- f(x)

This is a trivial example.  In the next few tutorial chapters, we will be examining the different kinds of derivation steps for making proofs and we will see more interesting examples of **judgments** and **proofs**.  For now, we simply note that **proof** display is similar to the way that the **expression** DAG is displayed using `expr_info`.  The "root" node is the judgment that is proven and is indexed as `0`.  In general, with exception to trivial examples like this one, it will have requirements supplied as a list of other indices (similarly to the **sub-expressions** in `expr_info`).

### 2 Additional Examples

As another simple example, consider the claim that $y$ is True if we know that $x$ is True and $x \Rightarrow y$ (an application of modus ponens).

As part of the example, we import `Variable` so we can give an example of defining the variables $x$ and $y$ from scratch, and we import the `Implies` expression constructor from proveit.logic. Then we prove $y$ as a result of the assumptions $x$ and $x \Rightarrow y$:

In [ ]:
from proveit import Variable
x = Variable('x')
y = Variable('y')
from proveit.logic import Implies
y_truth_from_modus_ponens = y.prove(assumptions={x, Implies(x, y)})

We can then look at the underlying proof, which is indeed a straightforward application of modus ponens:

In [ ]:
y_truth_from_modus_ponens.proof()

We can increase the complexity slightly, looking at a proof of $z$ given $x$, $x \Rightarrow y$, and $y \Rightarrow z$. The proof uses repeated application of modus ponens:

In [ ]:
from proveit import Variable, ExprTuple
from proveit.logic import Implies
x = Variable('x')
y = Variable('y')
z = Variable('z')
some_assumptions = ExprTuple(Implies(x, y), Implies(y, z), x)
claim = z
claim.prove(assumptions=some_assumptions).proof()

Proofs of more interesting claims will need to wait until later tutorial chapters. What might seem like simple applications of modus tollens, for example, actually involve derived theorems, `Forall` expressions, and `Forall` instantiations (universal instantiations), which are covered in later tutorials.

### Iterated assumptions

It is possible to include an `Iter` expression as an assumption.  When this is done, however, the assumed truth will automatically be wrapped in an `And` operation (as a logical conjunction).  The `Iter` itself is not a "truth".  The conjunction of the iteration is a truth.

In [ ]:
from proveit.core_expr_types._common_ import A_1_to_m

In [ ]:
A_1_to_m.prove([A_1_to_m])

In [ ]:
# This has the same effect:
from proveit.logic import And
And(A_1_to_m).prove([A_1_to_m]).proof()

In [ ]:
from proveit._common_ import A, B
from proveit.logic import And
And(A, B).prove([A, B]).proof()

### Default assumptions

If you do not supply a set of assumptions into the `prove` method (or any other **Prove-It** method that accepts an `assumptions` argument), it will use `proveit._core_.defaults.defaults.assumptions` (aliased as `proveit.defaults.assumptions`).  By default, this "default" is the empty set, but it can be changed for convenience.

For example, we can repeat our earlier proof of $f(x)$ by shifting the assumption into the default assumptions and calling `.prove()` without explicitly including the assumption:

In [ ]:
from proveit import defaults
defaults.assumptions = [fx]
new_fx_truth = fx.prove()
new_fx_truth

In [ ]:
new_fx_truth.proof()

In [ ]:
%end proof_basics

# Next chapter: <a href="tutorial03_implies.ipynb">Implication</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>